# Конфигурация

In [1]:
from pprint import pprint
from copy import deepcopy
from typing import Dict
from collections import Counter
import pickle
import lzma

from userknn import UserKnn
# import userknn

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import rectools 
from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.models import PopularModel
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import MAP, NDCG, Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics

from implicit.nearest_neighbours import CosineRecommender

In [2]:
K_RECOS = 10
RANDOM_SEED = 32

np.random.seed(RANDOM_SEED)

In [ ]:
metrics = {'MAP@1': MAP(k=1), 'MAP@5': MAP(k=5), 'MAP@10': MAP(k=10),
           'NDCG@1': NDCG(k=1), 'NDCG@5': NDCG(k=5), 'NDCG@10': NDCG(k=10),
           'Precision@1': Precision(k=1), 'Precision@5': Precision(k=5), 'Precision@10': Precision(k=10),
           'Recall@1': Recall(k=1), 'Recall@5': Recall(k=5), 'Recall@10': Recall(k=10),
           'MeanInvUserFreq@1': MeanInvUserFreq(k=1), 'MeanInvUserFreq@5': MeanInvUserFreq(k=5), 'MeanInvUserFreq@10': MeanInvUserFreq(k=10),
           'Serendipity@1': Serendipity(k=1), 'Serendipity@5': Serendipity(k=5), 'Serendipity@10': Serendipity(k=10)}

# Загрузка данных

In [3]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])

In [4]:
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

In [5]:
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    }, 
    inplace=True)

interactions = Interactions(interactions)

In [6]:
headtail(interactions.df)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0
5476246,648596,12225,2021-08-13,76.0,0.0
5476247,546862,9673,2021-04-13,2308.0,49.0
5476248,697262,15297,2021-08-20,18307.0,63.0
5476249,384202,16197,2021-04-19,6203.0,100.0
5476250,319709,4436,2021-08-15,3921.0,45.0


In [7]:
interactions.df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       float64       
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 208.9 MB


In [8]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

# Подготовка данных

производится Leave-time-out разбиение -- последняя неделя на test, остальное на train

In [15]:
max_date = interactions.df['datetime'].max()

train = interactions.df[(interactions.df['datetime'] < max_date - pd.Timedelta(days=7))]
test = interactions.df[(interactions.df['datetime'] >= max_date - pd.Timedelta(days=7))]

# оставляем только теплых пользователей в тесте
hot_test = test[test['user_id'].isin(train['user_id'].unique())]

catalog = train[Columns.Item].unique()

print(f"train: {train.shape}")
print(f"test: {test.shape}")
print(f"hot test: {hot_test.shape}")

train: (4985269, 5)
test: (490982, 5)
hot test: (349088, 5)


In [16]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=None,
    item_features_df=None
)

# Обучение userKNN

в качестве модели Recommender использовалась CosineRecommender c количеством соседей равным 30 (как на семинаре)

In [11]:
recommender = CosineRecommender(K=30)

In [12]:
userknn_model = UserKnn(recommender)

In [13]:
%%time
userknn_model.fit(train)

C:\Users\BIT\.conda\envs\RecSys\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.39046406745910645 seconds
  warnings.warn(


  0%|          | 0/896791 [00:00<?, ?it/s]

CPU times: total: 4h 32min 51s
Wall time: 28min 5s


In [41]:
# with lzma.open("user_knn.xz", "wb") as file:
#     pickle.dump(userknn_model, file)

In [11]:
with lzma.open("user_knn.xz", "rb") as file:
    userknn_model = pickle.load(file)

In [12]:
%%time
recos = userknn_model.predict(hot_test, N_recs=10)

CPU times: total: 44.8 s
Wall time: 1min 1s


# Раcчёт метрик для тёплых юзеров

In [24]:
hot_metric_values = calc_metrics(
    metrics,
    reco=recos,
    interactions=hot_test,
    prev_interactions=train,
    catalog=catalog,
)

In [25]:
pd.DataFrame(hot_metric_values, index=['userKNN'])

,Precision@1,Recall@1,Precision@5,Recall@5,Precision@10,Recall@10,NDCG@1,NDCG@5,NDCG@10,MAP@1,MAP@5,MAP@10,MeanInvUserFreq@1,MeanInvUserFreq@5,MeanInvUserFreq@10,Serendipity@1,Serendipity@5,Serendipity@10
userKNN,0.000755,0.000305,0.002413,0.005438,0.003897,0.017671,0.000755,0.001995,0.003176,0.000305,0.001736,0.003353,10.008312,8.646891,7.947655,0.000047,0.000066,0.000068


# Рекомендации для холодных пользователей в тесте (пункт 2 задание 1) (3 балла)

при отсутсвии юзера в train модель userKNN выдаёт ошибку, поэтому для выдачи рекомендаций таким юзерам будет использоваться popular модель

в сервисе юзер будет проверяться по наличию в маппинге userKNN, если он в нём отсутсвует, то будут выдаваться рекомендации popular моделью

In [28]:
cold_user_id = 9999999
hot_user_id = 663436

In [29]:
userknn_model.eval(cold_user_id)

KeyError: 9999999

In [24]:
pop = PopularModel()
pop.fit(dataset);

In [25]:
pop_recs = pop.recommend(
    users=dataset.user_id_map.external_ids,
    dataset=dataset,
    k=10,
    filter_viewed=False  # True - удаляет просмотренные айтемы из рекомендаций 
)

In [26]:
pop_recs = pop_recs.head(10)['item_id'].to_list()

In [27]:
# with open("popular_answer.pkl", "wb") as file:
#     pickle.dump(pop_recs, file)

In [30]:
with open("popular_answer.pkl", "rb") as file:
    pop_recs = pickle.load(file)

In [31]:
def predict_rec(user_id):
    if user_id in userknn_model.users_mapping:
        print('userKNN')
        ans = userknn_model.eval(user_id).item_id.to_list()
    else:
        print('popular')
        ans = pop_recs
    return ans

In [32]:
predict_rec(cold_user_id)

popular


[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

In [33]:
predict_rec(hot_user_id)

userKNN


[6939]

# Количество рекомендаций меньше N (пункт 2 задание 2) (3 балла)

Данную проблему можно решать следующими способами:
1) при получении от модели userKNN количества рекомендаций меньше N дополняем отсутсвующие айтемы популярным начиная с самого популярного
2) при определении похожих юзеров сделать цикл while в котором мы увеличивем количество рассматриваемых юзеров (параметр N в Recommender), который будет прекращаться при достижении нужного количества айтемов (k_recs)

Второй способ кажется более правильным, но у него узкие места:
1) большое количество операций merge для проверки достаточности количества айтемов для рекомендации (merge айтемов сходжих пользователей)
2) долгое дополнение айтемов для новых и слишком уникальных пользователей (малое количество айтемов у похожих пользователей) -- т.е для достижения нужного количества айтемов нужно много схожих пользователей

Поэтому в данном пункте был выбран первый способ с дополнением от модели popular

In [34]:
# количество юзеров в тесте с количеством рекомендаций меньше 10
recos.groupby('user_id').count().query('item_id < 10').shape[0]

50092

In [35]:
bad_user_id = 663436
k_recs = 10

In [36]:
ans = userknn_model.eval(bad_user_id).item_id.to_list()

In [37]:
ans, len(ans)

([6939], 1)

при в предикте userKNN оказался только 1 айтем

In [38]:
new_ans = ans + [item for item in pop_recs if item not in ans][:k_recs-len(ans)]

In [39]:
new_ans, len(new_ans)

([6939, 10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142], 10)

в результате получаем дополненный ответ userKNN популярным, при чём в дополнении популярным нужно удалять айтемы, которые уже есть в userKNN

при таком подходе мы всё равно ограничены лимитом выдачи popular модели, но по требованиям к рекомендациям сервиса их должно быть строго 10, и в крайнем случае можно расширить лимит выдачи popular модели с запасом